In [5]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from exp.nb_XLibrary import *

In [13]:
from pathlib import Path
import pickle

# Path and data

In [8]:
home_dir = Path('.').resolve()
data_dir = home_dir/'data'
file_path = data_dir/'imdb'
file_path

PosixPath('/Users/xianli/Desktop/fast/Part2/data/imdb')

In [9]:
il = TextList.from_files(file_path, include = ['train','test','unsup'])
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [10]:
proc_tok = TokenizeProcessor(max_workers=8)
proc_num = NumericalizeProcessor()

In [11]:
ll = label_by_func(sd, lambda x: 0, proc_x=[proc_tok, proc_num])

In [14]:
pickle.dump(ll, open(file_path/'ll_lm.pkl', 'wb'))
pickle.dump(proc_num.vocab, open(file_path/'vocab_lm.pkl', 'wb'))

In [ ]:
# loading
ll = pickle.load(open(path/'ll_lm.pkl', 'rb'))
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))

In [15]:
bs = 64
bptt = 70
data = lm_databunchify(ll, bs, bptt)

In [21]:
ll

SplitData
Train: LabeledData
x: TextList (90061 items)
[[2, 7, 1845, 7, 9669, 198, 7, 3755, 7, 4460, 220, 67, 375, 27, 2397, 124, 7, 182, 7, 5501, 2574, 17, 19, 1199, 507, 9, 7, 16, 22, 298, 335, 28, 80, 36154, 11, 7, 9669, 22, 149, 27, 7, 4230, 7, 18051, 9, 7, 151, 293, 172, 0, 14, 2805, 202, 74, 520, 64, 8, 29, 10, 16, 209, 9, 7, 8, 131, 28, 8, 29, 297, 1036, 10, 30, 43, 240, 8, 1447, 17, 168, 8, 31, 20, 16, 25, 37, 2041, 142, 26, 90, 26, 16, 96, 41, 99, 9, 7, 8, 212, 20, 16, 15, 16866, 47, 12, 269, 2766, 3357, 11, 672, 2766, 12139, 83, 39788, 200, 9, 7, 8, 29, 15, 298, 12, 0, 63, 28, 181, 69, 93, 437, 375, 47, 7, 9669, 10, 7, 4460, 10, 11, 7, 18051, 9, 3], [2, 18, 41, 130, 19, 29, 11, 18, 87, 37, 475, 28, 19, 29, 6603, 9, 18, 73, 37, 122, 59, 179, 14, 7, 1517, 107, 18, 58, 37, 53, 19, 668, 11, 113, 2012, 5856, 9, 18, 58, 37, 53, 14, 855, 710, 6603, 107, 18, 58, 37, 408, 80, 1077, 9, 7, 154, 73, 18, 159, 14, 7, 2125, 68, 18, 269, 159, 14, 7, 2578, 55, 8, 7, 2247, 7, 4974, 66, 7, 2578

In [22]:
dl = iter(data.train_dl)

In [25]:
x, y = next(dl)
x.shape, y.shape 

(torch.Size([64, 70]), torch.Size([64, 70]))

# AWD-LSTM

## LSTM
We need to implement those equations (where $\sigma$ stands for sigmoid):

![LSTM cell and equations](images/lstm.jpg)
(picture from [Understanding LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Chris Olah.)

If we want to take advantage of our GPU, it's better to do one big matrix multiplication than four smaller ones. So we compute the values of the four gates all at once. Since there is a matrix multiplication and a bias, we use `nn.Linear` to do it.

We need two linear layers: one for the input and one for the hidden state.

In [27]:
class LSTMCell(nn.Module):
    def __init__(self, ni, nh):
        super().__init__()
        self.ih = nn.Linear(ni, 4*nh) # the xU above
        self.hh = nn.Linear(nh, 4*nh) # tge hW above
    
    def forward(self, input, state):
        h, c = state
        gates = (self.ih(input) + self.hh(h)).chunk(4,1)
        input_gate, forget_gate, output_gate = map(torch.sigmoid, gates[:3])
        cell_gate = gates[3].tanh()
        c = forget_gate*c + input_gate*cell_gate
        h = output_gate*c.tanh()
        return h, (h, c)